### Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader



if (torch.cuda.is_available()):
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Supported Device: {device}")
from einops import rearrange

Supported Device: cpu


### Helper Functions

In [2]:
def proceess_data(data:str, keys:str) -> tuple:
    features = []
    lines = data.split("\n")
    for line in lines:
        features.append(list(map(float, line.split("\t"))))

    np_features = np.array(features).T[::-1,:]
    print(np_features)
    scaler = StandardScaler().fit(np_features)
    scaled_features =  scaler.transform(np_features)
    return scaler, scaled_features

### Data Processing  (Berkshire Hathaway Data)

In [3]:
data = """129619	126401	97322	103869	92774	76332	74803	67145	58535	79209	101760	85385	63822	118906	110518	94623	81506	53378	77745
23758	22435	22353	22566	25128	18602	21136	21718	16434	18125	20460	20027	18685	19435	19210	17923	19898	19172	19962
353842	318621	353409	328161	308793	306167	327662	390538	350719	310739	307942	282097	248027	245317	207454	180782	172757	220051	200516
29066	27496	27493	26403	28050	28714	17494	17596	17375	16658	16542	16533	17505	17152	17132	17583	17325	17535	17208
24681	24009	23530	23144	23208	22094	21877	21265	20751	20397	19900	19449	17527	18584	18175	17660	16280	17135	16807
199646	195214	193160	190181	181381	177850	177288	176722	176364	174123	173652	172789	159276	161539	159902	158522	152408	156556	154719
16947	16284	16028	15674	15584	15139	15125	15038	14918	14752	14659	14567	15065	14714	14887	14899	14298	14934	14741
84626	85652	85853	83502	78119	73395	73618	73822	73875	73770	73758	73695	81882	71865	71753	81659	81025	81228	81269
143743	143451	143020	142887	128308	125794	125369	124920	125475	123195	121646	115460	119827	117506	115631	110292	113361	112330	111168
31914	31173	30359	28657	26878	25727	25368	23512	25239	23732	23756	19782	22018	20992	21718	18093	20764	20113	20172
18556	19635	19937	22421	22305	22562	22673	22452	22409	22106	21709	20155	20975	20642	20193	18632	19422	19155	18918
124781	125347	123624	122744	116496	119081	119661	114262	114965	115223	114531	103368	107691	106689	104233	97490	102194	89907	97193
55077	52233	52525	50116	47971	47580	46005	46072	48132	45544	43693	42319	44026	40665	39614	37186	41189	40588	39807
34473	35140	35156	35167	35190	35204	35586	35592	35603	35635	35630	35658	35621	35615	35619	35707	35612	35610	35622
569776	582543	546631	511602	493438	496126	539881	534421	494775	484431	456337	402493	408791	738654	352359	321112	373334	356846	342773
-74655	-73568	-72265	-67826	-64972	-63934	-62906	-59795	-53072	-45446	-39418	-8125	-24075	-14815	-9700	-3109	-5937	-5252	-4799
531477	517810	465600	468711	406470	408950	528921	450662	411379	418601	385702	347815	320001	267300	272000	339590	311832	318350\t301215
350.29	341	308.76	308.89	267.01	273.01	352.91	299	272.94	277.92	255.47	231.86	212.94	178.50	182.83	226.5	208.02	213.16\t200.89"""

In [4]:
keys = """Short-term investment in U.S. Treasury Bills
FIxed Maturity Securities
Equity Securities
Equity Method Investments
Loans Receiveables
Property, Plant and Equipment
Equipment held for lease/ Service model
Goodwill
Unpaid Losses
Unearned Premiums
Life, Annuity and Insuarance Benefits
Notes payable and other borrowings
Accounts payable
Capital in excess of par value
Retained earnings
Treasury stock, at cost
BRK-A
BRK-B"""

In [5]:
scaler, np_scaled_features = proceess_data(data, keys)
print(np_scaled_features.shape)

[[ 7.77450e+04  1.99620e+04  2.00516e+05  1.72080e+04  1.68070e+04
   1.54719e+05  1.47410e+04  8.12690e+04  1.11168e+05  2.01720e+04
   1.89180e+04  9.71930e+04  3.98070e+04  3.56220e+04  3.42773e+05
  -4.79900e+03  3.01215e+05  2.00890e+02]
 [ 5.33780e+04  1.91720e+04  2.20051e+05  1.75350e+04  1.71350e+04
   1.56556e+05  1.49340e+04  8.12280e+04  1.12330e+05  2.01130e+04
   1.91550e+04  8.99070e+04  4.05880e+04  3.56100e+04  3.56846e+05
  -5.25200e+03  3.18350e+05  2.13160e+02]
 [ 8.15060e+04  1.98980e+04  1.72757e+05  1.73250e+04  1.62800e+04
   1.52408e+05  1.42980e+04  8.10250e+04  1.13361e+05  2.07640e+04
   1.94220e+04  1.02194e+05  4.11890e+04  3.56120e+04  3.73334e+05
  -5.93700e+03  3.11832e+05  2.08020e+02]
 [ 9.46230e+04  1.79230e+04  1.80782e+05  1.75830e+04  1.76600e+04
   1.58522e+05  1.48990e+04  8.16590e+04  1.10292e+05  1.80930e+04
   1.86320e+04  9.74900e+04  3.71860e+04  3.57070e+04  3.21112e+05
  -3.10900e+03  3.39590e+05  2.26500e+02]
 [ 1.10518e+05  1.92100e+04 

### KT Estimator

In [10]:
class KTEstimator():
    def __init__(self, data:np.array = np_scaled_features, K = 4, T = 1, train_size:float = 0.80) -> None:
        self.K = K
        self.T = T
        self.istrained = False
        self.scaler = scaler
        offset = 1 # Manipulating train-test split to accomodate at least one slab of (K+T) observation in test-set
        train_set, test_set = data[:int(len(data)*train_size)-offset], data[int(len(data)*train_size)-offset:]  
        
        # Creating Train Feature - Label Matrix        
        train_D = np.zeros(shape=(train_set.shape[1], train_set.shape[0]-(K+T)+1, K))
        train_L = np.zeros(shape=(train_set.shape[0]-(K+T)+1, 2))
        for i in range(train_D.shape[1]):
            train_D[:,i,:] = train_set[i:i+K, :].T
            train_L[i] = train_set[i+K:i+K+T, -2:]
        
        # Creating Test Feature - Label Matrix        
        test_D = np.zeros(shape=(test_set.shape[1], test_set.shape[0]-(K+T)+1, K))
        test_L = np.zeros(shape=(test_set.shape[0]-(K+T)+1, 2))
        for i in range(test_D.shape[1]):
            test_D[:,i,:] = test_set[i:i+K, :].T
            test_L[i] = test_set[i+K:i+K+T, -2:]
        
        X_train = np.zeros(shape = (train_D.shape[1], train_set.shape[1] * K))
        X_test = np.zeros(shape = (test_D.shape[1], test_set.shape[1] * K))

        for i in range(X_train.shape[0]):
            X_train[i] = np.concatenate([arr for arr in train_D[:,i,:]])

        for i in range(X_test.shape[0]):
            X_test[i] = np.concatenate([arr for arr in test_D[:,i,:]])
        
        
        # Test Train Split
        X_train = torch.tensor(X_train, dtype = torch.float32, device = device, requires_grad=False)
        y_train = torch.tensor(train_L,dtype = torch.float32, device = device, requires_grad=False)
        X_test = torch.tensor(X_test,dtype = torch.float32, device = device, requires_grad=False)
        y_test = torch.tensor(test_L,dtype = torch.float32, device = device, requires_grad=False)

        self.X_train, self.X_test, self.y_train, self.y_test = X_train, X_test, y_train, y_test

        return None
    
    def fit(self, model, convergence:float = 1e-7, num_epochs:int = 2000, batch_size:int = 4, lr:float = 0.001):

        # Training the model
        loss_fn = nn.MSELoss()
        opt = torch.optim.Adam(model.parameters(), lr = lr)
        print_every = 100

        dataset = TensorDataset(self.X_train, self.y_train)
        train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        losses = []
        print(self.X_train, self.X_test)
        for epoch in range(num_epochs):
            l = 0
            for data, target in train_dataloader:
                opt.zero_grad()
                y_pred = model(data)
                loss = loss_fn(y_pred, target)
                l += loss.item()
                loss.backward()
                opt.step()
            
            losses.append(l/len(self.X_train))

            if epoch != 0 and abs(losses[-1] - losses[-2]) < convergence:
                print(f'Final Training Loss: {losses[-1]}')
                print(f'Converged after {epoch} epochs')
                break

            if epoch % print_every == 0:
                print(f"Epoch: {epoch} | Loss: {losses[-1]}")
        
        self.train_loss = losses
        self.istrained = True
        return None
    
    def predict(self, model):
        loss_fn = nn.MSELoss()
        # Trained Model Prediction
        with torch.no_grad():
            test_indices = np.arange(0, len(self.X_test)-(self.K + self.T))
            print(len(self.X_test), print(len(self.X_train)))
            for i in test_indices:
                y = model(self.X_test[i])
                loss = loss_fn(y, self.y_test[i])
                print(f"Prediction: {y}, Actual: {self.y_test[i]}, Loss: {loss}")
        return None

Multilayer Perceptron

In [7]:
class MLP(nn.Module):
  def __init__(self, input_dim, hidden_size1, hidden_size2, output_dim):
    super(MLP, self).__init__()
    self.lin1 = nn.Linear(input_dim, hidden_size1)
    self.lin2 = nn.Linear(hidden_size1, hidden_size2)
    self.lin3 = nn.Linear(hidden_size2, output_dim)

  def forward(self, x):
    y1 = self.lin1(x)
    y2 = torch.relu(y1)
    y3 = self.lin2(y2)
    y4 = torch.relu(y3)
    y = self.lin3(y4)
    return y

In [17]:
K = 4
T = 1
train_size = 0.70
train_set, test_set = np_scaled_features[:int(len(np_scaled_features)*train_size)], np_scaled_features[int(len(np_scaled_features)*train_size):]  
train_set.shape, test_set.shape

((13, 18), (6, 18))

In [27]:
# Creating Feature - Label Matrix        
train_D = np.zeros(shape=(train_set.shape[1], train_set.shape[0]-(K+T)+1, K))
train_L = np.zeros(shape=(train_set.shape[0]-(K+T)+1, 2))
for i in range(train_D.shape[1]):
    train_D[:,i,:] = train_set[i:i+K, :].T
    train_L[i] = train_set[i+K:i+K+T, -2:]
X_train = torch.tensor(train_D,dtype = torch.float32, device = device, requires_grad=False)
y_train = torch.tensor(train_L,dtype = torch.float32, device = device, requires_grad=False)
train_D, train_L

(array([[[-0.52745069, -1.65538979, -0.35335545,  0.25382546],
         [-1.65538979, -0.35335545,  0.25382546,  0.98959892],
         [-0.35335545,  0.25382546,  0.98959892,  1.37787623],
         [ 0.25382546,  0.98959892,  1.37787623, -1.17194103],
         [ 0.98959892,  1.37787623, -1.17194103, -0.17379802],
         [ 1.37787623, -1.17194103, -0.17379802,  0.58419446],
         [-1.17194103, -0.17379802,  0.58419446, -0.4596827 ],
         [-0.17379802,  0.58419446, -0.4596827 , -1.41667424],
         [ 0.58419446, -0.4596827 , -1.41667424, -1.01812063],
         [-0.4596827 , -1.41667424, -1.01812063, -0.66363475],
         [-1.41667424, -1.01812063, -0.66363475, -0.59285792]],
 
        [[-0.19148205, -0.5623874 , -0.22153008, -1.14879346],
         [-0.5623874 , -0.22153008, -1.14879346, -0.54454639],
         [-0.22153008, -1.14879346, -0.54454639, -0.43890879],
         [-1.14879346, -0.54454639, -0.43890879, -0.79103412],
         [-0.54454639, -0.43890879, -0.79103412, -0.

In [28]:
X_train = np.zeros(shape = (train_D.shape[1], train_set.shape[1] * K))
# X_test = np.zeros(shape = (test_D.shape[1], test_set.shape[1] * K))

for i in range(X_train.shape[0]):
    X_train[i] = np.concatenate([arr for arr in train_D[:,i,:]])
X_train

array([[-0.52745069, -1.65538979, -0.35335545,  0.25382546, -0.19148205,
        -0.5623874 , -0.22153008, -1.14879346, -1.33753052, -1.02798593,
        -1.77738966, -1.65022839, -0.67372214, -0.6082243 , -0.65028713,
        -0.59860994, -1.36076424, -1.23445679, -1.56370336, -1.03228783,
        -1.31336681, -1.18300162, -1.47737005, -1.04348177, -0.67074667,
        -0.36983793, -1.36143357, -0.42440687,  0.63990399,  0.63156045,
         0.59024971,  0.71926944, -1.19613412, -1.09100198, -0.99772208,
        -1.27539037, -1.01333384, -1.0281443 , -0.86472722, -1.53521418,
        -1.28185527, -1.118868  , -0.93524943, -1.47854034, -1.41965009,
        -2.14002573, -0.92519516, -1.39028534, -1.13423731, -0.97241528,
        -0.84788898, -1.67730458,  0.585971  ,  0.54642056,  0.5530123 ,
         0.86611996, -1.292155  , -1.15360011, -0.99126849, -1.50541709,
         1.27343347,  1.2568891 ,  1.23187169,  1.33515526, -1.11182892,
        -0.90490479, -0.98361687, -0.64840824, -1.1

In [11]:
K = 4
T = 1
train_size = 0.7
est = KTEstimator(np_scaled_features, K, T, train_size)
hidden_size1 = 30
hidden_size2 = 10

model = MLP(input_dim = est.X_train.shape[1], hidden_size1 = hidden_size1 , hidden_size2 = hidden_size2, output_dim = 2)

#Trained Model Prediction
convergence = 1e-7
num_epochs = 2000
batch_size = 4
lr = 0.01
est.fit(model, convergence, num_epochs, batch_size, lr)
est.predict(model)

tensor([[-0.5275, -1.6554, -0.3534,  0.2538, -0.1915, -0.5624, -0.2215, -1.1488,
         -1.3375, -1.0280, -1.7774, -1.6502, -0.6737, -0.6082, -0.6503, -0.5986,
         -1.3608, -1.2345, -1.5637, -1.0323, -1.3134, -1.1830, -1.4774, -1.0435,
         -0.6707, -0.3698, -1.3614, -0.4244,  0.6399,  0.6316,  0.5902,  0.7193,
         -1.1961, -1.0910, -0.9977, -1.2754, -1.0133, -1.0281, -0.8647, -1.5352,
         -1.2819, -1.1189, -0.9352, -1.4785, -1.4197, -2.1400, -0.9252, -1.3903,
         -1.1342, -0.9724, -0.8479, -1.6773,  0.5860,  0.5464,  0.5530,  0.8661,
         -1.2922, -1.1536, -0.9913, -1.5054,  1.2734,  1.2569,  1.2319,  1.3352,
         -1.1118, -0.9049, -0.9836, -0.6484, -1.1155, -0.8884, -0.9835, -0.6414],
        [-1.6554, -0.3534,  0.2538,  0.9896, -0.5624, -0.2215, -1.1488, -0.5445,
         -1.0280, -1.7774, -1.6502, -1.2276, -0.6082, -0.6503, -0.5986, -0.6889,
         -1.2345, -1.5637, -1.0323, -0.8340, -1.1830, -1.4774, -1.0435, -0.9455,
         -0.3698, -1.3614, 

In [23]:
dataset = TensorDataset(est.X_train, est.y_train)

(tensor([[-0.5275, -1.6554, -0.3534, -0.1915, -0.5624, -0.2215, -1.3375, -1.0280,
          -1.7774, -0.6737, -0.6082, -0.6503, -1.3608, -1.2345, -1.5637, -1.3134,
          -1.1830, -1.4774, -0.6707, -0.3698, -1.3614,  0.6399,  0.6316,  0.5902,
          -1.1961, -1.0910, -0.9977, -1.0133, -1.0281, -0.8647, -1.2819, -1.1189,
          -0.9352, -1.4197, -2.1400, -0.9252, -1.1342, -0.9724, -0.8479,  0.5860,
           0.5464,  0.5530, -1.2922, -1.1536, -0.9913,  1.2734,  1.2569,  1.2319,
          -1.1118, -0.9049, -0.9836, -1.1155, -0.8884, -0.9835],
         [-1.6554, -0.3534,  0.2538, -0.5624, -0.2215, -1.1488, -1.0280, -1.7774,
          -1.6502, -0.6082, -0.6503, -0.5986, -1.2345, -1.5637, -1.0323, -1.1830,
          -1.4774, -1.0435, -0.3698, -1.3614, -0.4244,  0.6316,  0.5902,  0.7193,
          -1.0910, -0.9977, -1.2754, -1.0281, -0.8647, -1.5352, -1.1189, -0.9352,
          -1.4785, -2.1400, -0.9252, -1.3903, -0.9724, -0.8479, -1.6773,  0.5464,
           0.5530,  0.8661, -1.15